# Entfaltung: Hands-On

In diesem Hands-On sollen die in den letzten Vorlesungen vorgestellten Grundlagen der Entfaltung anhand eines einfachen Beispiels praktisch demonstriert werden.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm

In [2]:
%matplotlib widget

In [3]:
plt.rcParams['image.cmap'] = 'inferno'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.constrained_layout.use'] = True

## Daten einlesen und ansehen

Zunächst wird der "Trainings"-Datensatz eingelesen.

Training meint hier die Berechnung der Antwortmatrix und Detektionseffizienz. 

Diese enthalten sowohl die im echten Experiment verfügbaren Observablen als auch die Wahrheit,
die aus Simulation bekannt ist.

In [4]:
labels_train = pd.read_hdf('training.hdf5', 'labels')
observables_train = pd.read_hdf('training.hdf5', 'observables')

Die Labels in unseren Simulationen beinhalten die Energien und eine Parametrisierung des Tracks (mit den Komponenten $\phi$ und $d$). Zusätzlich existiert noch der Eintrag `noise`, der das Level des Hintergrundrauschens angibt. Dieser ist hier konstant auf 4.5 gesetzt und nicht weiter wichtig.

In [5]:
labels_train.head()

,E,phi,d,noise
0,1.110809,2.300641,-0.018607,4.5
1,1.768499,0.825967,-0.207421,4.5
2,3.900696,1.399377,-1.148939,4.5
3,2.352842,1.483698,-0.080060,4.5
4,1.527332,2.743621,-0.475558,4.5


In [6]:
observables_train.head()

,px_0_0,px_0_1,px_0_2,px_0_3,px_0_4,px_0_5,px_0_6,px_0_7,px_1_0,px_1_1,...,px_7_7,q_tot,cog_x,cog_y,conc,cov_xx,cov_yy,cov_xy,cov_phi,cov_a
0,11,5,6,5,1,4,9,5,6,6,...,7,322.0,-0.013310,-0.011535,0.034161,0.436000,0.453283,0.036226,0.902483,0.407399
1,3,3,4,5,5,3,7,3,5,4,...,5,372.0,0.050691,0.079109,0.032258,0.390891,0.388080,-0.026942,0.811457,0.416464
2,5,3,1,2,2,9,10,11,4,8,...,11,402.0,0.285714,0.022743,0.042289,0.426947,0.424399,-0.002335,1.035102,0.428333
3,2,5,7,4,14,7,2,3,4,5,...,0,374.0,-0.015279,-0.011459,0.048128,0.316148,0.431059,-0.023530,-1.376449,0.311516
4,4,6,6,5,5,2,5,7,6,6,...,4,327.0,-0.010048,-0.053735,0.039755,0.428221,0.386490,0.011131,-1.325768,0.431004


In [7]:
# the first 64 columns have the detector pixels
images_train = observables_train.iloc[:, :64].to_numpy().reshape((-1, 8, 8))

In [8]:
from ipywidgets import interact


fig, ax = plt.subplots()
plot = ax.imshow(images_train[0])
bar = fig.colorbar(plot)


def show_event(event=0):
    plot.set_data(images_train[event])
    plot.autoscale()
    bar.update_normal(plot)
    
interact(show_event, event=(0, len(images_train) - 1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='event', max=545662), Output()), _dom_classes=('widget-in…

<function __main__.show_event(event=0)>

Die Observablen sind einerseits die genauen Ladungswerte für alle Pixel (`px_i_j`), und andererseits eine Anzahl an Features, die im Anschluss an die Simulation anhand der Pixelwerte berechnet wurden. 

Wichtig ist hier zuächst nur die Gesamtladung $Q_\mathrm{tot}$. 

Diese ist besonders stark korreliert mit der ursprünglichen Energie $E$ der Einzelereignisse.

In [9]:
plt.figure()

# geomspace creates bins that are equidistant in log-space
bins_E = np.geomspace(labels_train.E.min(), labels_train.E.max(), 101)
bins_q_tot = np.geomspace(observables_train.q_tot.min(), observables_train.q_tot.max(), 101)

h = plt.hist2d(
    labels_train.E,
    observables_train.q_tot,
    bins=[bins_E, bins_q_tot], 
    norm=LogNorm(),
)

plt.xscale('log')
plt.yscale('log')

plt.xlabel('$E$')
plt.ylabel(r'$Q_\mathrm{tot}$')
plt.colorbar(h[3])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Wir lesen nun auch einen Test-Datensatz ein.

Gemessene Daten verfügen nicht wie die Simulationsdaten über Label/Wahrheit, d.h. es gibt nur Observablen.

Die Aufgabe der Entfaltung ist es nun die Verteilung einer Variablen nur durch Kenntnis über die Observablen und über den Detektionsprozess zu ermitteln.

Damit im Anschluss unsere Entfaltung dennoch mit einer Wahrheit verglichen werden kann, werden hier Simulationsdaten verwendet, die die Wahrheit dennoch enthalten.

In [10]:
observables_test = pd.read_hdf('test.hdf5', 'observables')
labels_test = pd.read_hdf('test.hdf5', 'labels')
images_test = observables_test.iloc[:, :64].to_numpy().reshape((-1, 8, 8))

Vergleicht man nun die Verteilungen der Observablen miteinander, so wird klar, dass das simulierte Energiespektrum des Trainingsdatensatzes nicht dem im Testdatensatz entspricht:

In [11]:
plt.figure()

# creates bins that are equidistant in log-space
bins = np.geomspace(observables_test.q_tot.min(), observables_test.q_tot.max(), 21)

plt.hist(observables_train.q_tot, bins=bins, density=True, histtype='step', label='Training')
plt.hist(observables_test.q_tot, bins=bins, density=True, histtype='step', label='Test')


plt.xlabel(r'$Q_\mathrm{tot}$')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Trigger

Um die Detektion von reinen Rausch-Ereignissen zu verringern, ist es notwendig eine Trigger-Bedingung an den Detektor zu stellen, d.h. das Ereignis muss eine gewisse Bedingung erfüllen um als Ereignis gewertet zu werden. Hier wird die Bedingung gestellt, dass für die Gesamtladung $Q_\mathrm{tot} > 385$ gilt. Dies sorgt dafür, dass Hintergrundereignisse weggeschnitten werden, aber auch, dass Ereignisse weggeworfen werden, insbesondere im Niederenergie-Bereich:

In [12]:
q_tot_cut = 385.0


triggered_train = observables_train.q_tot > q_tot_cut
triggered_test = observables_test.q_tot > q_tot_cut


print(f'all events:       {len(observables_train)}')
print(f'Q_tot_min:        {observables_train.q_tot.min()}')
print(f'triggered events: {np.count_nonzero(triggered_train)}')
print(f'ratio:            {np.count_nonzero(triggered_train) / len(observables_train):.1%}')

all events:       545663
Q_tot_min:        321.0
triggered events: 177638
ratio:            32.6%


In [13]:
threshold = 10
n_over_threshold = 10

def trigger(images, threshold, n_over_threshold):
    return np.count_nonzero(images > threshold, axis=(-2, -1)) >= n_over_threshold

triggered_train = trigger(images_train, threshold, n_over_threshold)
triggered_test = trigger(images_test, threshold, n_over_threshold)


print(f'all events:       {len(observables_train)}')
print(f'Q_tot_min:        {observables_train.q_tot.min()}')
print(f'triggered events: {np.count_nonzero(triggered_train)}')
print(f'ratio:            {np.count_nonzero(triggered_train) / len(observables_train):.1%}')

all events:       545663
Q_tot_min:        321.0
triggered events: 109377
ratio:            20.0%


In [14]:
plt.figure()

kwargs = dict(bins=bins_q_tot, histtype='step', )
plt.hist(observables_train.q_tot, label='All Events', **kwargs)
plt.hist(observables_train.q_tot[triggered_train], label='Triggered Events', **kwargs)
# plt.axvline(q_tot_cut, color='C1', label='trigger cut')

plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$Q_\mathrm{tot}$')

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plt.figure()

plt.hist(
    labels_train.E, bins=bins_E,
    histtype='step',
    label='Alle simulierten Ereignisse',
)

plt.hist(
    labels_train.E[triggered_train], bins=bins_E,
    histtype='step',
    label='Vom Trigger akzeptiert'
)

plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.xlabel('$E$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$E$')

Es ist von Interesse diese Effekte des Triggers auszugleichen. Bei echten Experimenten muss darüber hinaus von der entfalteten Zählrate auf den eigentlich gesuchten physikalischen Fluss geschlossen werden. Dieses Vorgehen wird als _Akzeptanzkorrektur_ bezeichnet.

# Definition des Binnings für die Entfaltung

Um die Entfaltung mit den hier behandelten Methoden durchzuführen ist es notwendig ein Binning, sowohl für die Zielvariable (hier die Energie), als auch die Observable (hier die Gesamtladung $Q_\mathrm{tot}$)  zu definieren.

In [16]:
n_bins_E = 27
n_bins_qtot = 60

binning_E = np.geomspace(3, 250, n_bins_E + 1)
binning_qtot = np.geomspace(450, 225e3, n_bins_qtot + 1)

## Underflow und Overflow Bins

Um den Fall einzuschließen, dass Ereignisse in keines der so definierten Bins fallen, sollte man Under- und Overflow-Bins definieren. Ein Ereignis liegt innerhalb eines Underflow-Bins, wenn es unterhalb der kleinsten Bingrenze liegt und innerhalb des Overflow-Bins, wenn es oberhalb der größten Bingrenze liegt.

In [17]:
def add_overflow(bins, positive=True):
    new = np.empty(len(bins) + 2)
    new[0] = 0 if positive else -np.inf
    new[1:-1] = bins
    new[-1] = np.inf
    return new

binning_E = add_overflow(binning_E)
binning_qtot = add_overflow(binning_qtot)

# Berechung der Migrationsmatrix

Die Einträge der Migrationsmatrix $\mathrm{A}$ beschreiben wie ein Ereignis aus einem Energie-Bin sich statistisch in Observablenbins wiederfinden wird. D.h, dass der Eintrag $A_{ij}$ die Wahrscheinlichkeit ist, dass ein Ereignis, das im Energiebin $j$ liegt, ebenfalls im Observablenbin $i$ liegt.

Diese Matrix ist eine sogenannte stochastische Matrix, d.h. dass

$$\sum_i A_{ij} = 1$$

Die Migrationsmatrix berechnet man, indem man ausgehend von Simulationen ein zweidimensionales Histogramm bzgl. der gewählten Observablen und Zielvariablen berechnet und dieses anschließend normiert.

In [18]:
observable_train = observables_train.q_tot[triggered_train]
energy_train = labels_train.E[triggered_train]

observable_test = observables_test.q_tot[triggered_test]
energy_test = labels_test.E[triggered_test]

In [19]:
H, *_ = np.histogram2d(
    observable_train,
    energy_train,
    bins=(binning_qtot, binning_E)
)

In [20]:
A = H / np.sum(H, axis=0)

In [21]:
plt.figure()
im = plt.pcolormesh(A)

plt.gca().set_aspect(1)
plt.xlabel('Energie Index')
plt.ylabel('$Q_\mathrm{tot}$ Index')
plt.colorbar(im)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Berechnung der Vektoren $\mathbf{g}$ und $\mathbf{f}$
Ähnlich können auch die Vektoren $\mathbf{g}$ (der Vektor, der die Bininhalte in den Observablenbins enthält) und $\mathbf{f}$ (der Vektor, der die Bininhalte in den Bins der Zielvariablen enthält) berechnet werden. Im Anschluss kann auch geprüft werden, ob die Entfaltungsgleichung 
$$A \cdot \mathbf{f} = \mathbf{g}$$
gültig ist.

In [22]:
g_train, _ = np.histogram(observable_train, binning_qtot)
f_train, _ = np.histogram(energy_train, binning_E)

g_test, _ = np.histogram(observable_test, binning_qtot)
f_test, _ = np.histogram(energy_test, binning_E)

f_train, f_test, g_test, g_test

(array([10317, 16201, 16051, 13657, 11309,  9101,  7161,  5755,  4560,
         3495,  2783,  2070,  1594,  1234,   966,   766,   570,   479,
          322,   258,   159,   145,   113,    81,    61,    45,    27,
           28,    69]),
 array([233, 389, 387, 374, 309, 250, 266, 440, 662, 238, 103,  98,  87,
         64,  55,  36,  30,  21,  15,  12,  13,   8,   9,   9,   6,   6,
          0,   2,  22]),
 array([288, 604, 463, 355, 279, 239, 188, 190, 158, 167, 156, 165, 183,
        141, 101,  55,  49,  32,  32,  24,  34,  25,  19,  21,  15,  10,
         11,  15,  14,  10,   4,   7,   4,   8,   7,   8,   2,   6,   4,
          4,   1,   3,   0,   1,   0,   4,   4,   5,   2,   3,   0,   2,
          1,   4,   2,   0,   1,   0,   0,   0,   2,  12]),
 array([288, 604, 463, 355, 279, 239, 188, 190, 158, 167, 156, 165, 183,
        141, 101,  55,  49,  32,  32,  24,  34,  25,  19,  21,  15,  10,
         11,  15,  14,  10,   4,   7,   4,   8,   7,   8,   2,   6,   4,
          4,   1,   3

In [23]:
plt.figure()

plt.stairs(g_train, lw=5, label='$g$')
plt.stairs(A @ f_train, label='$A \cdot f$')

plt.legend()
plt.yscale('log')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Berechnung der Akzeptanz

Wie zuvor angekündigt ist es interessant die Trigger-bedingten Akzeptanz-Effekte herauszurechnen. Hierzu wird der Vektor $\mathbf{f}$ mit allen Events ohne Berücksichtigung des Triggers berechnet und einmal mit Trigger. Das Verhältnis der einzelnen Einträge dieser Vektoren ist dann die Akzeptanz in diesem Fall.

In [24]:
energy_all = labels_train.E.values
f_untriggered, _ = np.histogram(energy_all, binning_E)

In [25]:
acceptance = f_train / f_untriggered

In [26]:
acceptance

array([0.02529985, 0.53154631, 0.66419763, 0.71803365, 0.76226746,
       0.78019717, 0.80154466, 0.82901181, 0.85425253, 0.86467095,
       0.90327816, 0.91877497, 0.92566783, 0.96481626, 0.982706  ,
       0.98966408, 0.9947644 , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        ])

In [27]:
e_center = 0.5 * (binning_E[:-1] + binning_E[1:])


plt.figure()
plt.stairs(acceptance[1:-1], binning_E[1:-1])
plt.xscale('log')
plt.xlabel('$E$')
plt.ylabel('Akzeptanz')
plt.axhline(1.0, ls=':', color='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Lösung der Entfaltungsgleichung

## Naiver Ansatz: Pseudo-Inverse

Der naivste Ansatz zur Lösung der Entfaltungsgleichung ist die Pseudoinverse:

$$\mathrm A \cdot \mathbf f = \mathbf g \Longleftrightarrow \mathbf f = \mathrm A^+ \mathbf g$$

In [28]:
def plot_unfolding_result(unfolded_hist, legend_title=None):
    fig, ax = plt.subplots()
    
    ax.stairs(unfolded_hist[1:-1], binning_E[1:-1], label='Entfaltung')
    ax.stairs((f_test / acceptance)[1:-1], binning_E[1:-1], label='Wahrheit')
    
    ax.legend(title=legend_title)
    
    ax.set_xscale('log')
    ax.set_yscale('log')
    
    ax.set_xlabel('E')
    ax.set_ylim(0.5, None)
    
    return fig, ax

In [29]:
np.linalg.pinv?

Signature: np.linalg.pinv(a, rcond=1e-15, hermitian=False)
Docstring:
Compute the (Moore-Penrose) pseudo-inverse of a matrix.

Calculate the generalized inverse of a matrix using its
singular-value decomposition (SVD) and including all
*large* singular values.

.. versionchanged:: 1.14
   Can now operate on stacks of matrices

Parameters
----------
a : (..., M, N) array_like
    Matrix or stack of matrices to be pseudo-inverted.
rcond : (...) array_like of float
    Cutoff for small singular values.
    Singular values less than or equal to
    ``rcond * largest_singular_value`` are set to zero.
    Broadcasts against the stack of matrices.
hermitian : bool, optional
    If True, `a` is assumed to be Hermitian (symmetric if real-valued),
    enabling a more efficient method for finding singular values.
    Defaults to False.

    .. versionadded:: 1.17.0

Returns
-------
B : (..., N, M) ndarray
    The pseudo-inverse of `a`. If `a` is a `matrix` instance, then so
    is `B`.

Raises
--

In [30]:
f_est_naive = np.linalg.pinv(A) @ g_test / acceptance

plot_unfolding_result(f_est_naive)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='E'>)

Deutlich zu sehen sind die starken Oszillationen, die das Ergebnis unbrauchbar machen. 

## Die Likelihood der Entfaltung

Die Bininhalte der Observablenbins sind Zählraten, somit können wir annehmen, dass diese einer Poisson-Statistik folgen:

$$f(g_j|\lambda_j) = \frac{\lambda_j^{g_j}}{g_j!} e^{-\lambda_j}$$

Der Erwartungswert $\lambda_j$ der einzelnen Observablen-Bininhalte lautet gemäß Entfaltungsgleichung

$$\lambda_j = \sum_i A_{ij} f_i = (\mathrm A \cdot \mathbf f)_j$$

Die Likelihood lautet entsprechend

$$\mathcal{L} = \prod_j f(g_j|\lambda_j) = \prod_j \frac{e^{-(\mathrm A \cdot \mathbf f)_j} (\mathrm A \cdot \mathbf f)_j^{g_j}}{g_j!}$$

Aus numerischen Gründen wird im Folgenden stets der negative Logarithmus der Likelihood verwendet:

$$-\log \mathcal{L} = \sum_j \left[(\mathrm A \cdot \mathbf f)_j - g_j\log(\mathrm A \cdot \mathbf f)_j\right]$$

Ebenfalls müssen wir verhindern, dass der Minimierer ungültige Funktionswerte erhält. Statt `nan` oder `inf` sollten immer sehr große, aber valide Werte zurückgegeben werden.

Insbesondere für den Fall das einzelne "Ausreißer" eine Wahrscheinlichkeit von "0" zugeordnet bekommen, verhindert dies vollständig die Minimierung, falls diese nicht durch eine sehr kleine, aber endliche Wahrscheinlichkeit ersetzt werden:

In [31]:
eps = 1e-30
np.log(0), np.log(eps)

<ipython-input-31-f3ed73188ef9>:2: RuntimeWarning: divide by zero encountered in log
  np.log(0), np.log(eps)


(-inf, -69.07755278982137)

In [32]:
def llh_poisson(f, A, g):
    
    # force positive f
    if np.any(f < 0):
        return np.finfo(np.float32).max
    
    lambda_ = A @ f
    
    eps = 1e-30 # to avoid log of 0
    return np.sum(lambda_ - g * np.log(lambda_ + eps))

Um eine solche Likelihood zu minimieren, kann man numerische Minimierer verwendet, so z.B. `scipy.optimize.minimize` oder wie zuletzt `iminuit`:

In [33]:
from iminuit import Minuit
from functools import partial

In [34]:
# use pinv unfolding as initial guess, replacing very small values
x0 = np.clip(np.linalg.pinv(A) @ g_test, 10, np.inf)


m = Minuit(
    partial(llh_poisson, A=A, g=g_test),
    x0,
    # generate names for our parameters
    name=[f"f_{i}" for i in range(n_bins_E + 2)]
)

m.errordef = m.LIKELIHOOD
m.limits = [(0, None) for _ in range(n_bins_E + 2)]

m.migrad()
m.hesse()
m.minos()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.79e+04                  │             Nfcn = 21436             │
│ EDM = 0.000171 (Goal: 0.0001)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │       SOME Parameters at limit       │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ f_0  │    200    │    60     │    -60     │     60     │    0    │         │       │
│ 1 │ f_1  │    470    │    100    │    -100    │    100     │    0    │         │       │
│ 2 │ f_2  │    390    │    90     │    -90     │     90     │    0    │         │       │
│ 3 │ f_3  │    360    │    80     │    -80     │     80     │    0    │         │       │
│ 4 │ f_4  │    270    │    70     │    -70     │     70     │    0    │         │       │
│ 5 │ f_5  │    270    │    70     │    -70     │     70     │    0    │         │       │
│ 6 │ f_6  │    240    │    70     │    -70     │     70     │    0    │         │       │
│ 7 │ f_7  │    440    │    70     │    -70     │     70     │    0    │         │       │
│ 8 │ f_8  │    700    │    60     │    -60     │     60     │    0    │         │       │
│ 9 │ f_9  │    220    │    40     │    -40     │     50     │    0    │         │       │
│ 10│ f_10 │    81     │    33     │    -33     │     34     │    0    │         │       │
│ 11│ f_11 │    119    │    35     │    -34     │     36     │    0    │         │       │
│ 12│ f_12 │    103    │    32     │    -31     │     32     │    0    │         │       │
│ 13│ f_13 │    24     │    25     │    -24     │     26     │    0    │         │       │
│ 14│ f_14 │    88     │    26     │    -25     │     27     │    0    │         │       │
│ 15│ f_15 │    25     │    18     │    -17     │     19     │    0    │         │       │
│ 16│ f_16 │    3.3    │   13.0    │    -3.3    │    17.6    │    0    │         │       │
│ 17│ f_17 │    34     │    17     │    -16     │     17     │    0    │         │       │
│ 18│ f_18 │    21     │    14     │    -13     │     15     │    0    │         │       │
│ 19│ f_19 │    14     │    11     │    -10     │     13     │    0    │         │       │
│ 20│ f_20 │  1.1e-6   │1491714.7e-6│  -1.1e-6   │2493007.1e-6│    0    │         │       │
│ 21│ f_21 │  1.3e-6   │1921159.5e-6│  -1.3e-6   │3309223.2e-6│    0    │         │       │
│ 22│ f_22 │    28     │    10     │    -10     │     10     │    0    │         │       │
│ 23│ f_23 │     7     │     6     │     -5     │     7      │    0    │         │       │
│ 24│ f_24 │    10     │     7     │     -6     │     8      │    0    │         │       │
│ 25│ f_25 │     5     │     5     │     -5     │     7      │    0    │         │       │
│ 26│ f_26 │  1.0e-6   │1065843.0e-6│  -1.0e-6   │1584117.8e-6│    0    │         │       │
│ 27│ f_27 │    0.4    │    7.6    │    -0.4    │    7.6     │    0    │         │       │
│ 28│ f_28 │    19     │     5     │     -5     │     5      │    0    │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────┬───────────────────────┬───────────────────────┬──────────────────────

In [35]:
plot_unfolding_result(m.values / acceptance)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='E'>)

### Regularisierung

Zuletzt fügen wir die Annahme hinzu, dass unsere Lösung _glatt_ sein soll. Dazu definieren wir eine Matrix, die die numerische zweite Ableitung repräsentiert:

$$C = \begin{bmatrix}
1 & -1 & 0 & \dots \\
-1 &  2 & -1 & \dots \\
0 & -1 & \ddots & & \vdots & \vdots \\
\vdots & \vdots & & \ddots & -1 & 0 \\
& & \dots & -1 & 2 & -1\\
& & \dots & 0 & -1 & 1\\
\end{bmatrix}
$$

In [36]:
from functools import lru_cache

@lru_cache()
def C_matrix(n):
    I = np.eye(n)
    C = 2.0 * I - np.roll(I, 1) - np.roll(I, -1)
    return C

In [37]:
C_matrix(5)

array([[ 1., -1.,  0.,  0.,  0.],
       [-1.,  2., -1.,  0.,  0.],
       [ 0., -1.,  2., -1.,  0.],
       [ 0.,  0., -1.,  2., -1.],
       [ 0.,  0.,  0., -1.,  1.]])

In [38]:
%timeit C_matrix(5)

72.9 ns ± 2.72 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Eine besonders glatte Kurve, so kann man annehmen, soll nun über eine kleine zweite Ableitung verfügen, d.h. wir addieren den Term

$$\mathcal{L}_\mathrm{tik} = \tau \sum_j |C_{ji} f_i|^2 = \tau \ \mathbf f^\top \mathrm C^\top \mathrm C \ \mathbf f$$
zur bestehenden Likelihood hinzu. Den offenen Parameter $\tau$ bezeichnen wir dabei als Regularisierungsstärke. Je größer er ist, desto stärker wird Nicht-Glattheit bestraft.

Da die erwartete Flusskurve hier einem Potenzgesetz folgt, kann die Bedingung, dass die zweite Ableitung möglichst klein werden muss besonders gut in der doppelt-logarithmischen Darstellung gerechtfertigt werden: Das Potenzgesetz wird hier zu einem linearen Zusammenhang:

$$\mathcal{L}_\mathrm{tik} = \tau \sum_j |C_{ji} f_i|^2 = \tau \log (\mathbf f)^\top \mathrm C^\top \mathrm C \log (\mathbf f)$$

In [39]:
def tikhonov_reg(f, tau, acceptance): 
    eps = 1e-30
    
    # we ignore under and overflow for the regularization
    C = C_matrix(len(f) - 2)
    
    # we regularize on the log of f with acceptance correction,
    # since only that is expected to be flat
    flat_f = np.log(f[1:-1] / acceptance[1:-1] + eps)
    return tau * np.sum((C @ flat_f) ** 2)


def llh_poisson_tikhonov(f, A, g, tau, acceptance):  
    if np.any(f < 0):
        return np.finfo(np.float32).max
        
    return llh_poisson(f, A, g) + tikhonov_reg(f, tau, acceptance)

In [40]:
tau_regularization = 0.1

In [41]:
# use pinv unfolding as initial guess
x0 = np.clip(np.linalg.pinv(A) @ g_test, 10, np.inf)

m = Minuit(
    partial(llh_poisson_tikhonov, A=A, g=g_test, tau=tau_regularization, acceptance=acceptance),
    x0,
    name=[f"f_{i}" for i in range(n_bins_E + 2)]
)

m.errordef = m.LIKELIHOOD
m.limits = [(0, None) for _ in range(n_bins_E + 2)]

m.migrad()
m.hesse()
m.minos()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = -1.789e+04                 │             Nfcn = 29138             │
│ EDM = 0.000175 (Goal: 0.0001)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ f_0  │    200    │    60     │    -60     │     60     │    0    │         │       │
│ 1 │ f_1  │    470    │    90     │    -90     │     90     │    0    │         │       │
│ 2 │ f_2  │    400    │    80     │    -80     │     80     │    0    │         │       │
│ 3 │ f_3  │    360    │    70     │    -70     │     70     │    0    │         │       │
│ 4 │ f_4  │    280    │    60     │    -60     │     60     │    0    │         │       │
│ 5 │ f_5  │    260    │    60     │    -60     │     60     │    0    │         │       │
│ 6 │ f_6  │    240    │    60     │    -60     │     60     │    0    │         │       │
│ 7 │ f_7  │    440    │    60     │    -60     │     70     │    0    │         │       │
│ 8 │ f_8  │    700    │    60     │    -60     │     60     │    0    │         │       │
│ 9 │ f_9  │    210    │    40     │    -40     │     40     │    0    │         │       │
│ 10│ f_10 │    89     │    28     │    -27     │     30     │    0    │         │       │
│ 11│ f_11 │    117    │    30     │    -29     │     31     │    0    │         │       │
│ 12│ f_12 │    91     │    26     │    -25     │     28     │    0    │         │       │
│ 13│ f_13 │    44     │    19     │    -17     │     20     │    0    │         │       │
│ 14│ f_14 │    71     │    20     │    -20     │     21     │    0    │         │       │
│ 15│ f_15 │    28     │    14     │    -12     │     15     │    0    │         │       │
│ 16│ f_16 │    15     │     9     │     -8     │     11     │    0    │         │       │
│ 17│ f_17 │    27     │    12     │    -11     │     13     │    0    │         │       │
│ 18│ f_18 │    23     │    11     │     -9     │     12     │    0    │         │       │
│ 19│ f_19 │     9     │     7     │     -6     │     9      │    0    │         │       │
│ 20│ f_20 │    2.7    │    2.3    │    -1.7    │    3.3     │    0    │         │       │
│ 21│ f_21 │    3.9    │    3.0    │    -2.3    │    4.0     │    0    │         │       │
│ 22│ f_22 │    20     │     9     │     -8     │     9      │    0    │         │       │
│ 23│ f_23 │    12     │     6     │     -5     │     7      │    0    │         │       │
│ 24│ f_24 │     9     │     5     │     -4     │     6      │    0    │         │       │
│ 25│ f_25 │    3.4    │    3.2    │    -2.4    │    4.3     │    0    │         │       │
│ 26│ f_26 │    0.9    │    1.1    │    -1.1    │    1.8     │    0    │         │       │
│ 27│ f_27 │    0.4    │    0.8    │    -0.8    │    3.1     │    0    │         │       │
│ 28│ f_28 │    19     │     5     │     -5     │     5      │    0    │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌──────────┬───────────────────────┬───────────────────────┬───────────────────────┬─

In [42]:
plot_unfolding_result(m.values / acceptance)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='E'>)

Natürlich kann diese Regularisierung auch auf die SVD Entfaltung angewendet werden. Dabei ergibt sich der Lösungsvector

$f = (A^TA + \tau\, C^TC)^{-1} A^T \mathbf g$

und im Vergleich zu der naiv invertierten Lösung oszilliert das Ergebnis schon deutlich weniger.

In [43]:
A_inv = np.linalg.pinv(A.T @ A + tau_regularization * C_matrix(A.shape[1]).T @ C_matrix(A.shape[1])) @ A.T

plot_unfolding_result(A_inv @ g_test / acceptance)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='E'>)

## Variation des Regularisierungsparameters

Die Entfaltung enthält den Parameter $\tau$, über den zunächst nichts weiter bekannt ist. Man 
kann feststellen, dass der Parameter die Glattheit der Lösung der Entfaltung bestimmt. Der kleine Resonanz-Peak in den Test-Daten bspw. löst sich abhängig von $\tau$ verschieden gut auf:

In [44]:
def unfold(tau):
    m = Minuit(
        partial(llh_poisson_tikhonov, A=A, g=g_test, tau=tau, acceptance=acceptance),
        x0,
        name=[f"f_{i}" for i in range(n_bins_E + 2)]
    )

    m.errordef = m.LIKELIHOOD
    m.limits = [(0, None) for _ in range(n_bins_E + 2)]

    m.migrad()
    m.hesse()
    
    return m

In [45]:
plot_unfolding_result(unfold(0.5).values / acceptance)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:xlabel='E'>)

### Minimum der globalen Korrelation

Eine Möglichkeit, einen "optimalen" Regularisierungsparameter zu bestimmen, ist die globale Korrelation des Ergebnisses zu minimieren.

`iminuit` liefert mithilfe der Hesse-Matrix die Covarianz-Matrix $\mathrm{Cov}(f)$ des Entfaltungsergebnisses.

Hieraus lässt sich die Globale Korrelation jedes Eintrags berechnen:

$$
\rho_j = \sqrt{1 - \mathrm{Cov}(f)_{jj} \cdot (\mathrm{Cov}(f)^{-1})_{jj}}
$$

Und der Mittelwert über alle Einträge wird minimiert durch einen "groben" Scan über verschiedene $\tau$

In [46]:
taus = np.geomspace(0.01, 10, 100)

def mean_global_correlation(cov):
    cov_inv = np.linalg.inv(cov)
    return np.mean(
        np.sqrt(1 - 1 / (np.diag(cov) * np.diag(cov_inv)))
    )


results = []
x0 = np.clip(np.linalg.pinv(A) @ g_test, 10, np.inf)

for tau in tqdm(taus):
    results.append(unfold(tau))
    
global_correlations = [mean_global_correlation(m.covariance) for m in results]

  0%|          | 0/100 [00:00<?, ?it/s]

In [47]:
plt.figure(constrained_layout=True)
plt.plot(taus, global_correlations, 'o')

plt.xlabel(r'$\tau$')
plt.ylabel('Mean Correlation')
plt.xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
best_idx = np.argmin(global_correlations)
best_tau = taus[best_idx]
best_fit = results[best_idx]

fig, ax =plot_unfolding_result(best_fit.values / acceptance)
ax.set_title(rf'Unfolding with $\tau={best_tau:.2f}$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Unfolding with $\\tau=0.43$')

# Abschätzung der Unsicherheiten

Wie immer in der Physik sind auch hier Messungen ohne Kenntnis über die Unsicherheiten von geringem Nutzen.

Oben haben wir bereits die Kovarianzmatrix der Fit-Parameter über die Hesse-Matrix aus imiuit bekommen.
Ebenso können wieder Likelihood-Scans mit der `minos` Methode durchgeführt werden.

Alternativ implementieren wir hier (hauptsächlich zur Wiederholung) einen MCMC-Ansatz.


Für das MCMC kann man eine simple Implementierung des Metropolis-Algorithmus verwenden. Als Resultat erhalten wir so eine Anzahl von Stichproben aus der Likelihood (die a-posteriori-Verteilung). Diese kann man nutzen um mittels eines Bayesianischen Ansatzes Aussagen über die Unsicherheit der Schätzung zu treffen.

In [50]:
def mcmc(x0, fun, step_size=1.5, n_steps=10000, print_acceptance=False):
    x = [x0]
    f = [fun(x0)]
    acc = 0
   
    for _ in tqdm(range(n_steps)):
        x_new = x[-1] + step_size * np.random.randn(len(x0))
        f_new = fun(x_new)
    
        prop_eval = -np.log(np.random.rand()) > (f_new - f[-1])
        
        if prop_eval:
            x.append(x_new)
            f.append(f_new)
            acc += 1
        else:
            x.append(x[-1])
            f.append(f[-1])
            
    if print_acceptance:
        print(f'{acc / n_steps:.2%} of proposed steps accepted.')
        
    return np.array(x), np.array(f)

In [51]:
from functools import partial

function = partial(llh_poisson_tikhonov, A=A, g=g_test, tau=0.43, acceptance=acceptance)

m = Minuit(
    function,
    x0,
    name=[f"f_{i}" for i in range(n_bins_E + 2)]
)

m.errordef = m.LIKELIHOOD
m.limits = [(0, None) for _ in range(n_bins_E + 2)]

m.migrad()
m.hesse()

    
x_sample, f_sample = mcmc(np.array(m.values), function, step_size=0.05 * f_test, n_steps=100000, print_acceptance=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

40.26% of proposed steps accepted.


Die Unsicherheit des Entfaltungsergebnisses kann nun bspw. durch die Angabe von Quantilen visualisiert werden.

(Dies ist nicht die einzige Möglichkeit, siehe Vorlesung)

In [52]:
from scipy.stats import norm

norm(0, 1).cdf(1)

0.8413447460685429

In [53]:
plt.figure()

plt.fill_between(
    np.arange(n_bins_E + 1),
    (np.quantile(x_sample, norm.cdf(-1), axis=0) / acceptance)[1:],
    (np.quantile(x_sample, norm.cdf(1), axis=0) / acceptance)[1:],
    alpha=0.2,
    step='post'
)


plt.stairs(
    (np.median(x_sample, axis=0) / acceptance)[1:-1],
    label='Median'
)


plt.stairs((f_test / acceptance)[1:-1], label='Wahrheit')
plt.stairs((m.values / acceptance)[1:-1], label='Entfaltung')
plt.legend()
plt.yscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
plt.figure()

bins_samples = np.geomspace(x_sample.min(), x_sample.max(), 101)

h = plt.hist2d(
    np.tile(np.arange(n_bins_E + 2), (len(x_sample), 1)).flatten(),
    x_sample.flatten(),
    bins=(np.arange(n_bins_E + 2), bins_samples),
    norm=LogNorm()
)

plt.stairs(f_test, label='Wahrheit')
plt.stairs(m.values, label='Entfaltung')

plt.yscale('log')
plt.colorbar(h[3])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
bin_idx = 8

plt.figure()
plt.hist(x_sample[:, bin_idx], bins=np.geomspace(x_sample[:, bin_idx].min(), x_sample[:, bin_idx].max(), 100))
plt.axvline(f_test[bin_idx], color='C1')
plt.xscale('log')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
bin1_idx = 9

plt.figure()
plt.scatter(
    x_sample[:, bin_idx],
    x_sample[:, bin_idx + 1],
    s=5, alpha=0.1, linewidth=0,
)
plt.xscale('log')
plt.yscale('log')


plt.matshow(np.corrcoef(x_sample.T), cmap='RdBu_r', vmin=-1, vmax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …